In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from google.colab import files
uploaded = files.upload()

Saving clothes_explicit.csv to clothes_explicit.csv
Saving clothes_explicit_links.csv to clothes_explicit_links.csv


In [3]:
clothes = pd.read_csv('clothes_explicit.csv')
clothes_links = pd.read_csv('clothes_explicit_links.csv')


In [5]:
clothes_df = pd.merge(clothes, clothes_links, on=['product_id'], how='inner')
clothes_df = clothes_links[clothes_links["product_id"].isin(clothes["product_id"])]
clothes_df = clothes_df.set_index('product_id').drop(columns=['Unnamed: 0'])
clothes_df.head()

,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features
product_id,,,,,,,,,,,
5,[{'https://static.zara.net/photos///2022/V/2/1...,women,TOPS_BODYSUITS,kabuki brush,short smooth densebristle brush ideal apply fo...,Top,other,short,other,short,short short
6,[{'https://static.zara.net/photos///2022/I/2/1...,women,ACCESSORIES,eyelash curler,allow eyelash curl raise single movement inten...,Accessories,other,other,other,other,NaN
7,[{'https://static.zara.net/photos///2022/I/2/1...,women,SHOES,eye brush,compact flat eye brush specially design apply ...,Shoes,other,other,other,other,NaN
8,[{'https://static.zara.net/photos///2022/I/2/1...,women,ACCESSORIES,eye brush,small eye brush design blend apply eye shadow ...,Accessories,other,other,other,other,NaN
9,[{'https://static.zara.net/photos///2023/V/2/1...,women,ACCESSORIES,makeup sponge,makeup sponge specially design liquid cream fo...,Accessories,other,other,water,other,water


# Preprocessing

In [24]:
clothes_df['material'].unique()

array(['other', 'knit', 'viscose', 'denim', 'faux leather', 'polyamide',
       'satin', 'satin viscose', 'linen viscose', 'linen', 'cotton',
       'nylon', 'cotton poplin', 'faux suede', 'poplin', 'organza',
       'semisheer', 'satiny', 'ramie', 'gabardine', 'cotton linen',
       'gabardine gabardinestyle', 'lyocell', 'knit linen',
       'linen lyocell', 'leather', 'faux', 'cotton jute linen', 'wool',
       'knit viscose', 'cashmere knit wool', 'knit wool', 'rubberise',
       'suede', 'cashmere knit', 'cotton knit', 'alpaca knit wool',
       'knit poplin', 'knit semisheer', 'denim knit',
       'organza satin semisheer', 'patent', 'polyurethane',
       'leather polyurethane', 'faux patent', 'polyurethane rubberise',
       'denim jute', 'velvet', 'neoprene', 'leather suede',
       'denim polyurethane rubberise', 'semisheer viscose',
       'gabardine gabardinetype', 'cotton knit lyocell silk',
       'cotton knit linen', 'cotton knit silk', 'cotton denim', 'acetate',
       '

In [25]:
clothes_df = clothes_df.fillna("").astype(str)

def describe_item(row):
    name = row.get("product_name", "").strip()
    material = row.get("material", "").strip() or "unspecified material"
    coverage = row.get("coverage_feature", "").strip() or "medium"

    sen = f"This is a {name}"
    if material != 'other':
      sen += f" made of {material}"
    if coverage != 'other':
      sen += f" with {coverage} coverage"
    return sen

clothes_df["text_complex"] = clothes_df.apply(describe_item, axis=1)

In [26]:
def apply_weather_rules(text):
    if any(word in text for word in ["wool", "fleece", "knit"]):
        text += ". It is suitable for cold weather."

    elif any(word in text for word in ["linen", "crop", "sleeveless", "tank", "silk", "acetate jute", "viscose", "short"]):
        text += ". It is suitable for warm or hot weather."

    elif any(word in text for word in ["denim", "cotton", "long", "leather"]):
        text += ". It is suitable for mild or cool weather."

    elif any(word in text for word in ["gabardine", "trench coat"]):
        text += ". It is suitable for rainy weather."
    return text

clothes_df["text_complex"] = clothes_df["text_complex"].apply(apply_weather_rules)

In [27]:
clothes_df.iloc[-60,-1]

'This is a stripe viscose shirt made of viscose with short coverage. It is suitable for warm or hot weather.'

In [28]:
for cat in clothes_df["category_type"].unique():
    var_name = cat.replace(" ", "_").lower() + "_df"
    globals()[var_name] = clothes_df[clothes_df["category_type"] == cat].copy()

In [29]:
accessories_df

,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex
product_id,,,,,,,,,,,,
6,[{'https://static.zara.net/photos///2022/I/2/1...,women,ACCESSORIES,eyelash curler,allow eyelash curl raise single movement inten...,Accessories,other,other,other,other,,This is a eyelash curler
8,[{'https://static.zara.net/photos///2022/I/2/1...,women,ACCESSORIES,eye brush,small eye brush design blend apply eye shadow ...,Accessories,other,other,other,other,,This is a eye brush
9,[{'https://static.zara.net/photos///2023/V/2/1...,women,ACCESSORIES,makeup sponge,makeup sponge specially design liquid cream fo...,Accessories,other,other,water,other,water,This is a makeup sponge
1168,[{'https://static.zara.net/photos///2023/I/0/1...,women,ACCESSORIES,chain link coin necklace,necklace feature metal link different size rai...,Accessories,other,necklace pendant,other,other,necklace pendant,This is a chain link coin necklace
1169,[{'https://static.zara.net/photos///2023/I/0/1...,women,ACCESSORIES,leather belt square buckle,leather belt square buckle loop line material,Accessories,leather,belt,other,other,leather belt,This is a leather belt square buckle made of l...
...,...,...,...,...,...,...,...,...,...,...,...,...
4124,[{'https://static.zara.net/photos///2023/I/0/2...,men,ACCESSORIES,training sock,pack pair seamless mesh sock rib trim,Accessories,other,other,other,other,,This is a training sock
4132,[{'https://static.zara.net/photos///2023/I/0/2...,men,ACCESSORIES,baseball training cap,peak cap adjustable strap backzara athleticz,Accessories,other,cap,other,other,cap,This is a baseball training cap
4136,[{'https://static.zara.net/photos///2023/I/0/2...,men,ACCESSORIES,strappy cycling culotte,culotte highly stretchy fabric elastic strap f...,Accessories,other,other,other,other,,This is a strappy cycling culotte


In [30]:
bottom_df.head()

,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex
product_id,,,,,,,,,,,,
88,[{'https://static.zara.net/photos///2023/I/0/1...,women,TROUSERS,highwaist culotte trouser,highwaist trouser feature pocket rear welt poc...,Bottom,other,short trouser,other,short,short trouser short,This is a highwaist culotte trouser with short...
92,[{'https://static.zara.net/photos///2023/I/0/1...,women,TROUSERS,carrot fit trouser dart detail,midwaist trouser viscose blend belt loop pocke...,Bottom,viscose,belt trouser,other,other,viscose belt trouser,This is a carrot fit trouser dart detail made ...
94,[{'https://static.zara.net/photos///2023/I/0/1...,women,SHORTS_SKORTS,trf highrise mom fit denim bermuda short,highrise mom fithighrise bermuda short fivepoc...,Bottom,denim,bermuda denim short,other,short,denim bermuda denim short short,This is a trf highrise mom fit denim bermuda s...
98,[{'https://static.zara.net/photos///2023/I/0/1...,women,JEANS,highrise stove pipe trf jean,highwaist slim ankle lengthhighwaist jean five...,Bottom,other,jean,other,other,jean,This is a highrise stove pipe trf jean
1281,[{'https://static.zara.net/photos///2023/I/0/1...,women,SHORTS_SKORTS,highwaist bikini,highwaist bikini,Bottom,other,bikini,other,other,bikini,This is a highwaist bikini


In [31]:
shoes_df.shape

(452, 12)

In [32]:
top_df.shape

(1087, 12)

# Zero-Shot Classification

In [33]:
#load the model
import torch
from transformers import pipeline
import tensorflow as tf

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [34]:
#candidate labels
labels = ["cold weather",
          "cool weather",
          "mild weather",
          "warm weather",
          "hot weather",
          "rainy weather"]

### Classifying Accessories Category

In [35]:
#zero-shot-classification on accessories category
# accessories_df = accessories_df.fillna("").astype(str)
# accessories_df_copy = accessories_df[accessories_df['combined_features'].str.strip() != ""].copy()
# accessories_df['text'] = accessories_df['combined_features'] + ' ' + accessories_df['product_name']
# texts = accessories_df['text'].tolist()

complex_texts = accessories_df["text_complex"].tolist()

batch_size = 16
results = []

for i in range(0, len(complex_texts), batch_size):
    batch_texts = complex_texts[i:i+batch_size]
    batch_results = classifier(batch_texts, candidate_labels=labels,
                               hypothesis_template="This outfit is designed to be worn in {} conditions.",
                               multi_label=True)

    for item in batch_results:
        results.append(item['labels'][0])

accessories_df['weather_label'] = results
accessories_df.head()

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
product_id,,,,,,,,,,,,,
6,[{'https://static.zara.net/photos///2022/I/2/1...,women,ACCESSORIES,eyelash curler,allow eyelash curl raise single movement inten...,Accessories,other,other,other,other,,This is a eyelash curler,warm weather
8,[{'https://static.zara.net/photos///2022/I/2/1...,women,ACCESSORIES,eye brush,small eye brush design blend apply eye shadow ...,Accessories,other,other,other,other,,This is a eye brush,mild weather
9,[{'https://static.zara.net/photos///2023/V/2/1...,women,ACCESSORIES,makeup sponge,makeup sponge specially design liquid cream fo...,Accessories,other,other,water,other,water,This is a makeup sponge,mild weather
1168,[{'https://static.zara.net/photos///2023/I/0/1...,women,ACCESSORIES,chain link coin necklace,necklace feature metal link different size rai...,Accessories,other,necklace pendant,other,other,necklace pendant,This is a chain link coin necklace,mild weather
1169,[{'https://static.zara.net/photos///2023/I/0/1...,women,ACCESSORIES,leather belt square buckle,leather belt square buckle loop line material,Accessories,leather,belt,other,other,leather belt,This is a leather belt square buckle made of l...,mild weather


In [36]:
accessories_df[accessories_df['weather_label'] == 'mild weather']

,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
product_id,,,,,,,,,,,,,
8,[{'https://static.zara.net/photos///2022/I/2/1...,women,ACCESSORIES,eye brush,small eye brush design blend apply eye shadow ...,Accessories,other,other,other,other,,This is a eye brush,mild weather
9,[{'https://static.zara.net/photos///2023/V/2/1...,women,ACCESSORIES,makeup sponge,makeup sponge specially design liquid cream fo...,Accessories,other,other,water,other,water,This is a makeup sponge,mild weather
1168,[{'https://static.zara.net/photos///2023/I/0/1...,women,ACCESSORIES,chain link coin necklace,necklace feature metal link different size rai...,Accessories,other,necklace pendant,other,other,necklace pendant,This is a chain link coin necklace,mild weather
1169,[{'https://static.zara.net/photos///2023/I/0/1...,women,ACCESSORIES,leather belt square buckle,leather belt square buckle loop line material,Accessories,leather,belt,other,other,leather belt,This is a leather belt square buckle made of l...,mild weather
1176,[{'https://static.zara.net/photos///2023/I/0/1...,women,ACCESSORIES,pack faux pearl necklace,pack necklaceslong necklace faux pearl appliqu...,Accessories,faux,necklace,other,other,faux necklace,This is a pack faux pearl necklace made of faux,mild weather
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3316,[{'https://static.zara.net/photos///2023/I/1/2...,men,ACCESSORIES,embossed wallet,emboss wallet main compartment pocket note car...,Accessories,other,other,other,other,,This is a embossed wallet,mild weather
3318,[{'https://static.zara.net/photos///2023/I/1/2...,men,ACCESSORIES,card holder purse,card holder compartment wallet zip closure pla...,Accessories,other,other,other,other,,This is a card holder purse,mild weather
3338,[{'https://static.zara.net/photos///2023/V/1/2...,men,ACCESSORIES,nylon multipocket bag,handbag multiple detachable pocket central hoo...,Accessories,nylon,bag,other,other,nylon bag,This is a nylon multipocket bag made of nylon,mild weather


In [38]:
accessories_df.to_csv("final_accessories.csv", index=False)
files.download("final_accessories.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Classifying Shoes Category

In [39]:
#zero-shot-classification on shoes category
# shoes_df = shoes_df.fillna("").astype(str)

# shoes_df['text'] = shoes_df['combined_features'] + ' ' + shoes_df['product_name']
# texts = shoes_df['text'].tolist()

complex_texts = shoes_df["text_complex"].tolist()

batch_size = 16
results = []

for i in range(0, len(complex_texts), batch_size):
    batch_texts = complex_texts[i:i+batch_size]
    batch_results = classifier(batch_texts, candidate_labels=labels,
                               hypothesis_template="This outfit is designed to be worn in {} conditions.",
                               multi_label=True)

    for item in batch_results:
        results.append(item['labels'][0])

shoes_df['weather_label'] = results
shoes_df.head()

,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
product_id,,,,,,,,,,,,,
7,[{'https://static.zara.net/photos///2022/I/2/1...,women,SHOES,eye brush,compact flat eye brush specially design apply ...,Shoes,other,other,other,other,,This is a eye brush,mild weather
1305,[{'https://static.zara.net/photos///2023/I/0/1...,women,SHOES,necklace resin piece,denim necklace transparent resin piece,Shoes,denim,denim necklace,other,other,denim denim necklace,This is a necklace resin piece made of denim. ...,mild weather
1421,[{'https://static.zara.net/photos///2023/I/1/1...,women,SHOES,ballet flat buckle,ballet flat buckle strap instep round toesole ...,Shoes,other,other,other,other,,This is a ballet flat buckle,warm weather
1432,[{'https://static.zara.net/photos///2023/I/1/1...,women,SHOES,flat leather cage sandal,flat leather cage sandal wide strap buckle ank...,Shoes,leather,sandal,other,other,leather sandal,This is a flat leather cage sandal made of lea...,mild weather
1442,[{'https://static.zara.net/photos///2023/I/1/1...,women,SHOES,leather mule loafer buckle,leather flat mule loafer metallic buckle detai...,Shoes,leather,loafer mule,other,other,leather loafer mule,This is a leather mule loafer buckle made of l...,mild weather


In [40]:
shoes_df[shoes_df['weather_label'] == 'warm weather']

,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
product_id,,,,,,,,,,,,,
1421,[{'https://static.zara.net/photos///2023/I/1/1...,women,SHOES,ballet flat buckle,ballet flat buckle strap instep round toesole ...,Shoes,other,other,other,other,,This is a ballet flat buckle,warm weather
1482,[{'https://static.zara.net/photos///2023/I/1/1...,women,SHOES,ballet flat square toe buckle,ballet flat strap detail buckle square toesole...,Shoes,other,other,other,other,,This is a ballet flat square toe buckle,warm weather
1579,[{'https://static.zara.net/photos///2023/I/1/1...,women,SHOES,satin ballet flat bow detail,satin ballet flat bow detail round toesole hei...,Shoes,satin,bow,other,other,satin bow,This is a satin ballet flat bow detail made of...,warm weather
2092,[{'https://static.zara.net/photos///2023/I/1/1...,women,SHOES,technical running trainer,run trainers adjustable elastic drawstring pul...,Shoes,other,trainer,other,other,trainer,This is a technical running trainer,warm weather
2093,[{'https://static.zara.net/photos///2023/I/1/1...,women,SHOES,satin sandal rhinestone,satin highheel sandal thin strap rhinestone ro...,Shoes,satin,sandal,other,other,satin sandal,This is a satin sandal rhinestone made of satin,warm weather
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897,[{'https://static.zara.net/photos///2023/I/1/2...,men,SHOES,rubberise toe shoe,sneakerstyle shoe combination material finish ...,Shoes,rubberise,shoe,other,other,rubberise shoe,This is a rubberise toe shoe made of rubberise,warm weather
3908,[{'https://static.zara.net/photos///2023/I/1/2...,men,SHOES,chunky sandal,strappy sandal strap hookandloop strap fasten ...,Shoes,other,sandal,other,other,sandal,This is a chunky sandal,warm weather
4112,[{'https://static.zara.net/photos///2023/I/1/2...,men,SHOES,run sneaker,run sneaker spume foam technology guarantee li...,Shoes,other,other,lightness,other,lightness,This is a run sneaker,warm weather


In [42]:
shoes_df.to_csv("final_shoes.csv", index=False)
files.download("final_shoes.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Classifying Tops Category

In [43]:
#zero-shot-classification on top category
#top_df = top_df.fillna("").astype(str)

#top_df['text'] = top_df['product_name'] + ' ' + top_df['combined_features']
#texts = top_df['text'].tolist()

complex_texts = top_df["text_complex"].tolist()

batch_size = 16
results = []

for i in range(0, len(complex_texts), batch_size):
    batch_texts = complex_texts[i:i+batch_size]
    batch_results = classifier(batch_texts, candidate_labels=labels,
                               hypothesis_template="This item is typically worn in {}.",
                               multi_label=True)

    for item in batch_results:
        results.append(item['labels'][0])

top_df['weather_label'] = results
top_df.head()

,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
product_id,,,,,,,,,,,,,
5,[{'https://static.zara.net/photos///2022/V/2/1...,women,TOPS_BODYSUITS,kabuki brush,short smooth densebristle brush ideal apply fo...,Top,other,short,other,short,short short,This is a kabuki brush with short coverage. It...,warm weather
87,[{'https://static.zara.net/photos///2023/I/0/1...,women,SWEATERS_CARDIGANS,basic high neck knit sweater,sweater feature high neck long sleeve decorati...,Top,knit,sweater,knit sweater,long,knit sweater knit sweater long,This is a basic high neck knit sweater made of...,cold weather
89,[{'https://static.zara.net/photos///2023/I/0/1...,women,WAISTCOATS_GILETS,short tailor waistcoat,vneck waistcoat feature false welt pocket butt...,Top,other,short waistcoat,other,short,short waistcoat short,This is a short tailor waistcoat with short co...,warm weather
90,[{'https://static.zara.net/photos///2023/I/0/1...,women,BLAZERS,doublebreaste oversized blazer,loosefitte blazer feature lapel collar long sl...,Top,other,blazer lapel,other,long,blazer lapel long,This is a doublebreaste oversized blazer with ...,mild weather
93,[{'https://static.zara.net/photos///2023/I/0/1...,women,SHIRTS,oversized shirt,loosefitte collared shirt feature long sleeve ...,Top,other,shirt,other,long,shirt long,This is a oversized shirt with long coverage. ...,mild weather


In [44]:
top_df[top_df['weather_label'] == 'warm weather']

,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
product_id,,,,,,,,,,,,,
5,[{'https://static.zara.net/photos///2022/V/2/1...,women,TOPS_BODYSUITS,kabuki brush,short smooth densebristle brush ideal apply fo...,Top,other,short,other,short,short short,This is a kabuki brush with short coverage. It...,warm weather
89,[{'https://static.zara.net/photos///2023/I/0/1...,women,WAISTCOATS_GILETS,short tailor waistcoat,vneck waistcoat feature false welt pocket butt...,Top,other,short waistcoat,other,short,short waistcoat short,This is a short tailor waistcoat with short co...,warm weather
1186,[{'https://static.zara.net/photos///2023/I/0/1...,women,TOPS_BODYSUITS,surplice swimsuit metal piece,swimsuit feature removable cup surplice neckli...,Top,other,swimsuit,other,other,swimsuit,This is a surplice swimsuit metal piece,warm weather
1198,[{'https://static.zara.net/photos///2023/I/0/1...,women,TOPS_BODYSUITS,crochet choker fringe,crochet choker bead appliqué fringe braid tie ...,Top,other,other,other,other,,This is a crochet choker fringe,warm weather
1209,[{'https://static.zara.net/photos///2023/I/0/1...,women,TOPS_BODYSUITS,print linen blend scarf,scarf linen blend fray edgesmeasurement x cm x,Top,linen,other,other,other,linen,This is a print linen blend scarf made of line...,warm weather
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4127,[{'https://static.zara.net/photos///2023/I/0/2...,men,T-SHIRTS,comfort workout tshirt,tshirt lightweight elastic fabric direction ro...,Top,other,short tshirt,lightweight,short,short tshirt lightweight short,This is a comfort workout tshirt with short co...,warm weather
4129,[{'https://static.zara.net/photos///2023/I/0/2...,men,T-SHIRTS,rib tank,sleeveless wide round neckzara athleticz,Top,other,tank,other,sleeveless,tank sleeveless,This is a rib tank with sleeveless coverage. I...,warm weather
4135,[{'https://static.zara.net/photos///2023/I/0/2...,men,T-SHIRTS,texture compression tshirt,tshirt technical fabric high elasticity adapt ...,Top,other,tshirt,breathability,sleeveless,tshirt breathability sleeveless,This is a texture compression tshirt with slee...,warm weather


In [45]:
top_df.to_csv("final_top.csv", index=False)
files.download("final_top.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Classifying Bottoms Category

In [46]:
#zero-shot-classification on bottom category

# bottom_df = bottom_df.fillna("").astype(str)

# bottom_df['text'] = bottom_df['combined_features'] + ' ' + bottom_df['product_name']
# texts = bottom_df['text'].tolist()

complex_texts = bottom_df["text_complex"].tolist()

batch_size = 16
results = []

for i in range(0, len(complex_texts), batch_size):
    batch_texts = complex_texts[i:i+batch_size]
    batch_results = classifier(batch_texts, candidate_labels=labels,
                               hypothesis_template="This clothing is suitable for {} weather.",
                               multi_label=True)

    for item in batch_results:
        results.append(item['labels'][0])

bottom_df['weather_label'] = results
bottom_df.head()

,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
product_id,,,,,,,,,,,,,
88,[{'https://static.zara.net/photos///2023/I/0/1...,women,TROUSERS,highwaist culotte trouser,highwaist trouser feature pocket rear welt poc...,Bottom,other,short trouser,other,short,short trouser short,This is a highwaist culotte trouser with short...,warm weather
92,[{'https://static.zara.net/photos///2023/I/0/1...,women,TROUSERS,carrot fit trouser dart detail,midwaist trouser viscose blend belt loop pocke...,Bottom,viscose,belt trouser,other,other,viscose belt trouser,This is a carrot fit trouser dart detail made ...,warm weather
94,[{'https://static.zara.net/photos///2023/I/0/1...,women,SHORTS_SKORTS,trf highrise mom fit denim bermuda short,highrise mom fithighrise bermuda short fivepoc...,Bottom,denim,bermuda denim short,other,short,denim bermuda denim short short,This is a trf highrise mom fit denim bermuda s...,warm weather
98,[{'https://static.zara.net/photos///2023/I/0/1...,women,JEANS,highrise stove pipe trf jean,highwaist slim ankle lengthhighwaist jean five...,Bottom,other,jean,other,other,jean,This is a highrise stove pipe trf jean,cool weather
1281,[{'https://static.zara.net/photos///2023/I/0/1...,women,SHORTS_SKORTS,highwaist bikini,highwaist bikini,Bottom,other,bikini,other,other,bikini,This is a highwaist bikini,cool weather


In [47]:
bottom_df[bottom_df['weather_label'] == 'warm weather']

,product_images,gender,product_category,product_name,details,category_type,material,product_subtype,weather_feature,coverage_feature,combined_features,text_complex,weather_label
product_id,,,,,,,,,,,,,
88,[{'https://static.zara.net/photos///2023/I/0/1...,women,TROUSERS,highwaist culotte trouser,highwaist trouser feature pocket rear welt poc...,Bottom,other,short trouser,other,short,short trouser short,This is a highwaist culotte trouser with short...,warm weather
92,[{'https://static.zara.net/photos///2023/I/0/1...,women,TROUSERS,carrot fit trouser dart detail,midwaist trouser viscose blend belt loop pocke...,Bottom,viscose,belt trouser,other,other,viscose belt trouser,This is a carrot fit trouser dart detail made ...,warm weather
94,[{'https://static.zara.net/photos///2023/I/0/1...,women,SHORTS_SKORTS,trf highrise mom fit denim bermuda short,highrise mom fithighrise bermuda short fivepoc...,Bottom,denim,bermuda denim short,other,short,denim bermuda denim short short,This is a trf highrise mom fit denim bermuda s...,warm weather
1422,[{'https://static.zara.net/photos///2023/I/0/1...,women,TROUSERS,carrot fit trouser dart detail,midwaist trouser viscose blend belt loop pocke...,Bottom,viscose,belt trouser,other,other,viscose belt trouser,This is a carrot fit trouser dart detail made ...,warm weather
1431,[{'https://static.zara.net/photos///2023/I/0/1...,women,SKIRTS,long satin skirt limited edition,highwaist long skirt viscose fabric asymmetric...,Bottom,satin viscose,skirt,other,long,satin viscose skirt long,This is a long satin skirt limited edition mad...,warm weather
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4099,[{'https://static.zara.net/photos///2023/I/0/2...,men,SHORTS,training short,training short lightweight stretchy technical ...,Bottom,other,short,lightweight,short,short lightweight short,This is a training short with short coverage. ...,warm weather
4119,[{'https://static.zara.net/photos///2023/I/0/1...,men,SHORTS,training short,workout short lightweight stretchy technical f...,Bottom,other,short,lightweight thermoseale,short,short lightweight thermoseale short,This is a training short with short coverage. ...,warm weather
4128,[{'https://static.zara.net/photos///2023/I/0/2...,men,SHORTS,jogger bermuda short embroidery,bermuda short adjustable elasticate waistband ...,Bottom,other,bermuda short,other,short,bermuda short short,This is a jogger bermuda short embroidery with...,warm weather


In [49]:
bottom_df.to_csv("final_bottom.csv", index=False)
files.download("final_bottom.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Recommendation

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 6.76568970e-02  6.34958893e-02  4.87131067e-02  7.93049634e-02
   3.74480449e-02  2.65277340e-03  3.93749550e-02 -7.09847454e-03
   5.93613833e-02  3.15369889e-02  6.00981116e-02 -5.29051647e-02
   4.06067669e-02 -2.59308256e-02  2.98428144e-02  1.12693582e-03
   7.35148862e-02 -5.03819585e-02 -1.22386619e-01  2.37028375e-02
   2.97265332e-02  4.24768403e-02  2.56337766e-02  1.99517491e-03
  -5.69190979e-02 -2.71598268e-02 -3.29035521e-02  6.60248324e-02
   1.19007103e-01 -4.58791293e-02 -7.26214647e-02 -3.25840004e-02
   5.23413382e-02  4.50552590e-02  8.25299695e-03  3.67024392e-02
  -1.39415637e-02  6.53918907e-02 -2.64272504e-02  2.06358891e-04
  -1.36642987e-02 -3.62810269e-02 -1.95043534e-02 -2.89738569e-02
   3.94270420e-02 -8.84091035e-02  2.62426748e-03  1.36714121e-02
   4.83063087e-02 -3.11565865e-02 -1.17329188e-01 -5.11690304e-02
  -8.85287896e-02 -2.18962505e-02  1.42986495e-02  4.44167927e-02
  -1.34815276e-02  7.43392929e-02  2.66382527e-02 -1.98762529e-02
   1.79191

In [ ]:
temperature_mapping = {
            (5, 10): "5-10°C cold weather",
            (10, 15): "10-15°C cool weather",
            (15, 20): "15-20°C mild weather",
            (20, 25): "20-25°C warm weather",
            (25, 30): "25-30°C hot weather"
        }

In [ ]:
def temperature_to_label(temperature):
        """
        Convert numeric temperature to label
        """
        try:
            temp = float(temperature)
            for (low, high), label in temperature_mapping.items():
                if low <= temp <= high:
                    return label
            if temp < 5:
                return "5-10°C cold weather"
            elif temp > 30:
                return "25-30°C hot weather"
        except ValueError:
            raise ValueError(f"Invalid temperature: {temperature}. Please enter 5-30.")

In [ ]:
temperature = '25'
temperature_label = temperature_to_label(temperature)
temp_range = temperature_label.split(' ')[0]

In [ ]:
#load MoritzLaurer model
import torch
from transformers import pipeline
import tensorflow as tf

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [ ]:
# RANDOM SAMPLE - This is the key speed improvement!
sample_size = 5

#Sample from TOP category

if len(top_df) > sample_size:
    sample_df = top_df.sample(sample_size) #, random_state=42)
    print(f"🎯 Sampling {sample_size} random items from {len(top_df)} total Top items")
else:
    sample_df = top_df
    print(f"🎯 Using all {len(sample_df)} top items")



# Prepare texts from sampled items
texts = []
for idx, row in sample_df.iterrows():
    text = f"{row['product_name']}.{row['details']}"
    texts.append(text)



# Single batch classification (fast!)
print("⚡ Classifying sampled items in one batch...")
results = classifier(texts,
               candidate_labels=[temperature_label],
               hypothesis_template="This item is comfortable to wear when the weather is {}.",
               multi_label=False
              )


# Handle results
if isinstance(results, dict):
    scores = [results['scores'][0]]
else:
    scores = [result['scores'][0] for result in results]


# Get top K indices from sampled items
scores = np.array(scores)
top_indices = np.argsort(scores)[::-1][:5]
scores, top_indices


# Build results
top_recommendations = []
for i, idx in enumerate(top_indices, 1):
    item = sample_df.iloc[idx]
    top_recommendations.append({'rank': i,
                            'product_name': item['product_name'],
                            'details': item['details'],
                            'confidence': round(scores[idx], 3),
                            'temperature': f"{temperature}°C",
                            'temperature_range': temp_range,
                            'sampled_from_total': f"{len(sample_df)}/{len(clothes_df)} items"})
top_recommendations[0]['product_name']

🎯 Sampling 5 random items from 1620 total Top items
⚡ Classifying sampled items in one batch...


'faux leather bomber jacket'

In [ ]:
# RANDOM SAMPLE - This is the key speed improvement!
sample_size = 5

#Sample from BOTTOM category

if len(bottom_df) > sample_size:
    sample_df = bottom_df.sample(sample_size) #, random_state=42)
    print(f"🎯 Sampling {sample_size} random items from {len(bottom_df)} total bottom items")
else:
    sample_df = bottom_df
    print(f"🎯 Using all {len(sample_df)} bottom items")



# Prepare texts from sampled items
texts = []
for idx, row in sample_df.iterrows():
    text = f"{row['product_name']}.{row['details']}"
    texts.append(text)



# Single batch classification (fast!)
print("⚡ Classifying sampled items in one batch...")
results = classifier(texts,
               candidate_labels=[temperature_label],
               hypothesis_template="This item is comfortable to wear when the weather is {}.",
               multi_label=False
              )


# Handle results
if isinstance(results, dict):
    scores = [results['scores'][0]]
else:
    scores = [result['scores'][0] for result in results]


# Get top K indices from sampled items
scores = np.array(scores)
top_indices = np.argsort(scores)[::-1][:5]
scores, top_indices


# Build results
bottom_recommendations = []
for i, idx in enumerate(top_indices, 1):
    item = sample_df.iloc[idx]
    bottom_recommendations.append({'rank': i,
                            'product_name': item['product_name'],
                            'details': item['details'],
                            'confidence': round(scores[idx], 3),
                            'temperature': f"{temperature}°C",
                            'temperature_range': temp_range,
                            'sampled_from_total': f"{len(sample_df)}/{len(clothes_df)} items"})
bottom_recommendations[0]['product_name']

🎯 Sampling 5 random items from 599 total bottom items
⚡ Classifying sampled items in one batch...


'straight fit trouser button'

In [ ]:
# RANDOM SAMPLE - This is the key speed improvement!
sample_size = 5

#Sample from SHOES category

if len(shoes_df) > sample_size:
    sample_df = shoes_df.sample(sample_size) #, random_state=42)
    print(f"🎯 Sampling {sample_size} random items from {len(shoes_df)} total shoes items")
else:
    sample_df = shoes_df
    print(f"🎯 Using all {len(sample_df)} shoes items")



# Prepare texts from sampled items
texts = []
for idx, row in sample_df.iterrows():
    text = f"{row['product_name']}.{row['details']}"
    texts.append(text)



# Single batch classification (fast!)
print("⚡ Classifying sampled items in one batch...")
results = classifier(texts,
               candidate_labels=[temperature_label],
               hypothesis_template="This item is comfortable to wear when the weather is {}.",
               multi_label=False
              )


# Handle results
if isinstance(results, dict):
    scores = [results['scores'][0]]
else:
    scores = [result['scores'][0] for result in results]


# Get top K indices from sampled items
scores = np.array(scores)
top_indices = np.argsort(scores)[::-1][:5]
scores, top_indices


# Build results
shoes_recommendations = []
for i, idx in enumerate(top_indices, 1):
    item = sample_df.iloc[idx]
    shoes_recommendations.append({'rank': i,
                            'product_name': item['product_name'],
                            'details': item['details'],
                            'confidence': round(scores[idx], 3),
                            'temperature': f"{temperature}°C",
                            'temperature_range': temp_range,
                            'sampled_from_total': f"{len(sample_df)}/{len(clothes_df)} items"})
shoes_recommendations[0]['product_name']

🎯 Sampling 5 random items from 479 total shoes items
⚡ Classifying sampled items in one batch...


'multipiece hightop sneaker'

In [ ]:
# RANDOM SAMPLE - This is the key speed improvement!
sample_size = 5

#Sample from ACCESSORIES category

if len(accessories_df) > sample_size:
    sample_df = accessories_df.sample(sample_size) #, random_state=42)
    print(f"🎯 Sampling {sample_size} random items from {len(accessories_df)} total accessories items")
else:
    sample_df = accessories_df
    print(f"🎯 Using all {len(sample_df)} accessories items")



# Prepare texts from sampled items
texts = []
for idx, row in sample_df.iterrows():
    text = f"{row['product_name']}.{row['details']}"
    texts.append(text)



# Single batch classification (fast!)
print("⚡ Classifying sampled items in one batch...")
results = classifier(texts,
               candidate_labels=[temperature_label],
               hypothesis_template="This item is comfortable to wear when the weather is {}.",
               multi_label=False
              )


# Handle results
if isinstance(results, dict):
    scores = [results['scores'][0]]
else:
    scores = [result['scores'][0] for result in results]


# Get top K indices from sampled items
scores = np.array(scores)
top_indices = np.argsort(scores)[::-1][:5]
scores, top_indices


# Build results
accessories_recommendations = []
for i, idx in enumerate(top_indices, 1):
    item = sample_df.iloc[idx]
    accessories_recommendations.append({'rank': i,
                            'product_name': item['product_name'],
                            'details': item['details'],
                            'confidence': round(scores[idx], 3),
                            'temperature': f"{temperature}°C",
                            'temperature_range': temp_range,
                            'sampled_from_total': f"{len(sample_df)}/{len(clothes_df)} items"})
accessories_recommendations[0]['product_name']

🎯 Sampling 5 random items from 181 total accessories items
⚡ Classifying sampled items in one batch...


'pack noshow sock'

In [ ]:
labels

['5-10°C cold weather',
 '10-15°C cool weather',
 '15-20°C mild weather',
 '20-25°C warm weather',
 '25-30°C hot weather',
 'rain']

In [ ]:
# Save to a JSON file
import json

config = {
    "model": "facebook/bart-large-mnli",
    "labels": ['5-10°C cold weather',
               '10-15°C cool weather',
               '15-20°C mild weather',
               '20-25°C warm weather',
               '25-30°C hot weather',
               'rain'],
    "hypothesis_template": "This item is comfortable to wear when the weather is {}."
}

with open("zero_shot_config.json", "w") as f:
    json.dump(config, f, indent=4)

In [ ]:
files.download("zero_shot_config.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>